# Unzip the downloaded zip files

In [29]:
import os
import glob
import zipfile
import shutil

In [ ]:

# --- 1. Define source and destination folders ---
source_folder = r"C:\Users\stgadmin\Project\Data downloads\T100"       # Folder containing your .wip files
destination_folder = r"C:\Users\stgadmin\Project\Data downloads\T100_Extractedcsvs"  # Folder where CSVs will go --- CORRECT

print(f"Source folder: {source_folder}")
print(f"Destination folder: {destination_folder}")


# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# --- 2. Find all .zip files in the source folder ---
zip_files = glob.glob(os.path.join(source_folder, "*.zip"))

if not zip_files:
    print("No .wip files found in the source folder.")
else:
    print(f"Found {len(zip_files)} .wip file(s) to extract.")

    for zip_path in zip_files:
        zip_name = os.path.basename(zip_path)
        zip_base_name = os.path.splitext(zip_name)[0]
        print(f"\nProcessing: {zip_name}")

        # Temporary folder to extract the .wip file
        temp_extract_folder = os.path.join(source_folder, "__temp_extract__")
        os.makedirs(temp_extract_folder, exist_ok=True)

        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(temp_extract_folder)

            # Move CSV files to destination folder
            csv_files = glob.glob(os.path.join(temp_extract_folder, "*.csv"))
            for csv_file in csv_files:
                csv_ext = os.path.splitext(csv_file)[1]  # keep extension
                new_csv_name = f"{zip_base_name}{csv_ext}"  # use only zip name
                shutil.move(csv_file, os.path.join(destination_folder, new_csv_name))

            print(f"✅ {len(csv_files)} CSV file(s) moved to destination folder.")

        except zipfile.BadZipFile:
            print(f"❌ Error: {zip_name} is not a valid zip file.")
        except Exception as e:
            print(f"❌ Unexpected error while processing {zip_name}: {e}")
        finally:
            # Clean up temporary folder
            if os.path.exists(temp_extract_folder):
                shutil.rmtree(temp_extract_folder)

print("\nAll files processed.")

Source folder: C:\Users\stgadmin\Project\Data downloads\T100
Destination folder: C:\Users\stgadmin\Project\Data downloads\T100_Extractedcsvs
Found 3 .wip file(s) to extract.

Processing: T_T100_SEGMENT_ALL_CARRIER_20251022_145557_2025.zip
✅ 1 CSV file(s) moved to destination folder.

Processing: T_T100_SEGMENT_ALL_CARRIER_20251022_145823_2024.zip
✅ 1 CSV file(s) moved to destination folder.

Processing: T_T100_SEGMENT_ALL_CARRIER_20251022_150014_2023.zip
✅ 1 CSV file(s) moved to destination folder.

All files processed.


# Data Load to SQL Server

In [ ]:

import pandas as pd
import urllib
from sqlalchemy import create_engine, text

In [ ]:
# --- 1. Define folder containing CSVs ---
folder_path = r"C:\Users\stgadmin\Project\air-travel-dwh\Data\T100"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

if not csv_files:
    print("No CSV files found.")
else:
    print(f"Found {len(csv_files)} CSV file(s) in the folder.")

# --- 2. Database connection parameters ---
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=ICT-210-11;"
    "DATABASE=AirtravelBronze;"
    "Trusted_Connection=yes;"
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
)
connection_url = f"mssql+pyodbc:///?odbc_connect={params}"
engine_destination = create_engine(connection_url, fast_executemany=True)

# --- 3. Ensure log table exists ---
with engine_destination.begin() as conn:
    conn.execute(text("""
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'T100_Load_Log')
        BEGIN
            CREATE TABLE dbo.T100_Load_Log (
                FileName NVARCHAR(255) PRIMARY KEY,
                LoadDate DATETIME DEFAULT GETDATE()
            );
        END
    """))

# --- 4. Function to load a single CSV if not already loaded ---
def load_csv_to_sql(csv_file, table_name='T_100_MonthlyData', chunksize=50000):
    csv_name = os.path.basename(csv_file)
    with engine_destination.begin() as conn:
        # Check if file has been loaded
        result = conn.execute(
            text("SELECT COUNT(*) FROM dbo.T100_Load_Log WHERE FileName = :fname"),
            {"fname": csv_name}
        ).scalar()

        if result > 0:
            print(f"⚠️ File already loaded, skipping: {csv_name}")
            return

        # Load CSV in chunks
        print(f"\n📄 Loading new file: {csv_name}")
        for chunk in pd.read_csv(csv_file, chunksize=chunksize):
            chunk.to_sql(
                name=table_name,
                con=conn,
                if_exists='append',
                index=False
            )

        # Log the loaded file
        conn.execute(
            text("INSERT INTO dbo.T100_Load_Log (FileName) VALUES (:fname)"),
            {"fname": csv_name}
        )
        print(f"✅ File successfully inserted and logged: {csv_name}")

# --- 5. Process all CSVs ---
for csv_file in csv_files:
    load_csv_to_sql(csv_file)

print("\n All new CSV files have been processed and uploaded to SQL Server.")

Found 3 CSV file(s) in the folder.

📄 Loading new file: T_T100_SEGMENT_ALL_CARRIER_2023.csv


C:\Users\stgadmin\AppData\Local\Temp\ipykernel_19760\1695617586.py:50: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, chunksize=chunksize):


✅ File successfully inserted and logged: T_T100_SEGMENT_ALL_CARRIER_2023.csv

📄 Loading new file: T_T100_SEGMENT_ALL_CARRIER_2024.csv
✅ File successfully inserted and logged: T_T100_SEGMENT_ALL_CARRIER_2024.csv

📄 Loading new file: T_T100_SEGMENT_ALL_CARRIER_2025.csv


C:\Users\stgadmin\AppData\Local\Temp\ipykernel_19760\1695617586.py:50: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, chunksize=chunksize):


✅ File successfully inserted and logged: T_T100_SEGMENT_ALL_CARRIER_2025.csv

 All new CSV files have been processed and uploaded to SQL Server.
